In [35]:

# step 1 run this cell to import the library and access the gpt api
import json
import re
import faiss
import torch
import requests
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import re
# ----------------------- SETUP -----------------------


In [36]:
# step3 to call the gpt api
def get_response(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1600
    }
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
    except Exception as e:
        return f"Error: {e}"


In [315]:
# # step4 prompt 1
# def prompt_func1(history_conversation):
#     history_text = ""
#     for i, (question, reply) in enumerate(history_conversation.items(), start=1):
#         history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"
#     # print(history_text)
#     prompt2 = f"""
#     You are an intelligent assistant helping Indian users find the right local service or business. Your job is to extract details from the user’s query in the exact sequence:
#     The following clarifications have already taken place:{history_text}
#     Follow the steps in the sequential manner. Do not fill the value of one field into another, e.g., do not use area to infer city. If something is missing, ask the related clarification question until it's confirmed, and then proceed to the next step.
#     you must accurately extract three details:
#      Normalize the query: convert Hindi, Hinglish, or misspelled English into correct English.
#     -Step 1 – Category Extraction (LLM Instruction):
#     Extract the list of 5 most relevant keyword that match the service or product the user is asking for and available on the justdial platform.
#     Once the keywords extracted do not ask more specfic question related to category.
#     After confirmation please also confirm the city and area.
#     -Step 2 – City Extraction (Second Priority): The city in India where the service is required. If no city is provided, ask for it.
#     If travel-related query like 'I want to travel to Mumbai', ask 'From which city do you want to travel to Mumbai?'.
#     If multiple cities are mentioned:
#       For travel queries → determine starting city like if user query is I want to travel to kanpur from mumbai then city shoulb be mumbai;
#       For service queries → determine the service location city in which user want the service.
#     If the location is a locality that can be confidently mapped to a city, confirm with the user.
#     If the city is outside India, respond: 'Sorry for the inconvenience, but our services are currently available only in India.
#     Could you please share the name of an Indian city where you would like to request this service? I can help you with that.'
#     -Do not proceed to Step 3 until City is confirmed.
#     Step 3 – Area Extraction (Final Priority):
#     The locality, neighbourhood, colony, or smaller part of the city where they want the service. If only a city is provided, ask 'Which area in <city> are you searching in?'.
#     The area should not be the same as the city.
#     When the extracted area does not exist in the given city, use a similarity check to find and return the closest matching area from that city
#     If the area is written in Hindi, Hinglish, or misspelled English, convert it to correct English.
#     If area seems invalid or mismatched, ask the user to provide a correct locality.
#     - If all three details (category list, area, city) are confidently identified and specific and there is no missing value and all value is filled  then only return in this format:
#       Return only this JSON format (no explanation text):
#           {{"category": ["category1", "category2", "category3", "category4", "category5"], "area": "area_name", "city": "city_name"}}  
#     If any detail is missing or unclear, output a direct question to get that detail.
#     Examples: 'Are you looking for a train ticket agent, bus booking, or flight booking?', 'Could you please specify the city where you need this service?', 'Which area in Delhi are you searching in?'.
#     Special Handling / Edge Cases:
#     Travel Queries → Always ask both origin and destination if unclear, plus service type. Event-based Queries (e.g., 'Doctor near me') → Ask for both city and area.
#     Multiple Locations → Determine service city vs. starting point (travel).
#     Non-service Queries → Politely ask them to rephrase in terms of a local business/service. """
#     return prompt2

In [317]:
# step4 prompt 1
def prompt_func1(history_conversation):
    history_text = ""
    for i, (question, reply) in enumerate(history_conversation.items(), start=1):
        history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"
    # print(history_text)
    prompt2 = f"""
    You are an intelligent assistant helping Indian users find the right local service or business on the Justdial platform.  
    The following clarifications have already taken place with user:{history_text}
    Your task is to extract and return the following information from the user’s query:  
    1. A list of 5 most relevant categories that match the service or product the user is asking for.  
    2. The area name (neighbourhood, locality, colony, or sub-part of the city).  
    3. The city name in India where the service is needed.  
    Guidelines:  
    - Normalize the query (e.g., convert Hindi/Hinglish/misspellings into proper English).  
    -If all three details (category, area, city) are confidently identified and specific and there is no missing value and all value is filled  then only return in this format:
      Return only this JSON format (no explanation text):  
    {{"category": ["category1", "category2", "category3", "category4", "category5"], "area": "area_name", "city": "city_name"}}  
   If any detail is missing or unclear, output a direct question to get that detail.
    If any detail is missing or unclear, do not output JSON. Instead, ask a direct clarification question such as:
    "Could you please specify the city where you need this service?"
    "Which area in Delhi are you searching in?"
    "Are you looking for a train ticket agent, bus booking, or flight booking?"   
 """
    return prompt2

In [263]:
response = 'How can I help you?'
cnt =0
history_conversation = {}
a = response.lower()
first_user = input(f"Assistant: {response}\nUser: ")

Assistant: How can I help you?
User:  I am looking for wedding caterer


In [264]:
history_conversation[f'{cnt} {a}'] = first_user.lower()
cnt+=1

In [273]:
 prompt1 = prompt_func1(history_conversation)
response = get_response(prompt1)

In [274]:
response

'{"category": ["Wedding Caterers", "Catering Services", "Event Caterers", "Party Caterers", "Buffet Caterers"], "area": "Malad West", "city": "Mumbai"}'

In [272]:
user_input = input(f"Assistant: {response}\nUser: ")
history_conversation[f'{cnt} {response}'] = user_input
cnt+=1

Assistant: Could you please confirm that you need a wedding caterer in Malad West, Mumbai?
User:  yes


In [4]:
#step5 Prompt 2 to handle the threshold
def prompt_func2(history_conversation, cat, area_query, threshold_cat, threshold_area, cat1, area):
    # Build history text from past conversation
    history_text = ""
    for i, (question, reply) in enumerate(history_conversation.items(), start=1):
        history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"

    # Build cat_text and area_text based on the categories and areas provided
    cat_text = ""
    for i, category in enumerate(cat1, start=1):
        cat_text += f"{i}. {category}\n"

    area_text = ""
    for i, location in enumerate(area, start=1):
        area_text += f"{i}. {location}\n"

    # Construct the main prompt
    prompt2 = f"""
You are assisting a user in identifying a local business category and area based on their query.
Here is the conversation history:
{history_text}
From the initial understanding:
- **Category**: "{cat}"
- **Area**: "{area_query}"
After comparing with our internal database using FAISS similarity search:
- **Category similarity score**: {threshold_cat}
- **Area similarity score**: {threshold_area}

The threshold for category or area is below our confidence level of 0.8 or 0.9 , indicating that the provided information may be **vague**, **uncommon**, **misspelled**, or **ambiguous**.
Action Required:
- If **Category similarity score** score is below 0.8: Provide the top similar categories from the database, specifically using the `cat1` column (which contains potential categories from the database) and ask for clarification. Here are some similar categories:
{cat_text}

- If **Area similarity score** score is below 0.9: Provide the top similar areas from the database, specifically using the `area` column (which contains potential areas from the database) and ask for clarification. Here are some similar areas:
{area_text}

- Prioritize the item with the **lower threshold** (either category or area) for clarification.

Your task:
Generate **one simple follow-up question** based on the following:
- If the category is unclear: "Are you looking for [specific example of categories from {cat_text}]?"
- If the area is unclear: "Could you please tell me which locality or area within {area_text} you're searching in?"
- If both are unclear: "Could you clarify both the exact service you're looking for and the area within your city?"

Special Cases:
- Multiple locations mentioned: Ask the user to clarify the service location vs. starting point if related to travel.
- Ambiguous categories (e.g., “repair”, “booking”, “printing”): Ask for more specifics on the type.
- Non-service-related queries: Politely ask the user to rephrase or clarify their needs.

Return only the **follow-up question**. Do not include explanations or additional information.
"""
    return prompt2


In [214]:
# step6
def prompt_follow1(categories , cat_org):
    category_text = ""
    for cat in categories :
        category_text += f"{cat} , "
    # print(history_text)
    prompt2 = f"""
    This is the {cat_org} which is extracted from the usery queries.
    These are the Link categories {category_text}
    Your Task to ask the follow up question regarding the Link category.
    Example Are you also interesting in {category_text}?
    Use the Example as context do not return the question which look like same as the example.
    Give the follow up question grammarily correct . and simple ensure Link categories be in the question.
     """
    return prompt2

In [215]:
# Change the path of directory.
# ART_DIR = "/content/drive/MyDrive/justdial for whatsapp"
MODEL_NAME = "intfloat/multilingual-e5-base"

# Load metadata
# df_cat = pd.read_parquet(f"df_cat.parquet")
df_cat = pd.read_parquet("df_cat.parquet", engine="fastparquet")
df_area = pd.read_parquet("df_area1.parquet" , engine="fastparquet")

# Load FAISS indexes
index_cat = faiss.read_index(f"index_cat.faiss")
index_area = faiss.read_index(f"index_area1.faiss")

# Load model for query encoding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
mdl = AutoModel.from_pretrained(MODEL_NAME).to(device).eval()

@torch.inference_mode()
def e5_encode(texts, is_query=False, batch_size=64, max_length=64):
    prefix = "query: " if is_query else "passage: "
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = [prefix + str(t).strip() for t in texts[i:i+batch_size]]
        enc = tok(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(device)
        out = mdl(**enc).last_hidden_state[:, 0]
        out = torch.nn.functional.normalize(out, p=2, dim=1)
        vecs.append(out.cpu().numpy().astype("float32"))
    return np.vstack(vecs)

In [246]:
df_cat.rename(columns = {'categroy_name' : 'category_name'} , inplace=True)

In [247]:
df_cat.head()

,national_catid,category_name
0,10050521,Body Massage Centres
1,11010608,Massage Centres For Men
2,10272436,Interior Designers
3,10076456,Car Rental
4,10042600,Beauty Spas


In [217]:
def format_category(text):
    # Split into main category and explanation
    if "(" in text and ")" in text:
        main, explanation = text.split("(", 1)
        main = main.strip()
        explanation = "(" + explanation.strip()
        return f"Main category is {main} and this is category explanation {explanation} "
    else:
        # If no explanation given
        return f"Main category is {text.strip()}"
# Example
# category = "Tent Rental Services (where tents are rented for events)"
# output = format_category(category)
# print(output)


In [218]:
# step 8
# Search algorithm for the embedding model
def _safe_cols(df: pd.DataFrame, wanted_cols: list) -> list:
    return [c for c in wanted_cols if c in df.columns]

def search_categories(user_query: str, top_k: int = 5) -> pd.DataFrame:
    user_query = format_category(user_query)
    qv = e5_encode([user_query], is_query=True)
    D, I = index_cat.search(qv, top_k)
    res = df_cat.iloc[I[0]].copy()
    res["cosine_similarity"] = D[0]
    cols = _safe_cols(res, ['national_catid', 'category_name'])
    return res[cols + ["cosine_similarity"]]
def search_areas(user_query: str, top_k: int = 5) -> pd.DataFrame:
    user_query = user_query.lower()
    cleaned = re.sub(r"[^\w]", "", user_query)
    qv = e5_encode([cleaned], is_query=True)
    D, I = index_area.search(qv, top_k)
    res = df_area.iloc[I[0]].copy()
    res["cosine_similarity"] = D[0]
    cols = _safe_cols(res, ["main_area", "pincode", "city"])
    return res[cols + ["cosine_similarity"]]

In [219]:
def gpt_result(df_cat_res , cat_gpt):
    categories = ""
    for cat in df_cat_res['category_name']:  # <-- fixed typo here
        categories += f"{cat}, "
    prompt = f"""
    System
You are an expert in Indian local services. 
{cat_gpt}
These are the options of categories {categories} , 
Search the category which is most similar the main category and fullfill the user needs and also used fuzzy logic or string matching with semantic search to find the most similar.
Task
Return exactly one string which must be a member of {categories}. No other text. 
    """
    return prompt , categories
    

In [220]:
# step 9
def extract_json(response):
    match = re.search(r'\{.*?\}', response, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None


In [221]:
def preprocess_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # keep only letters, numbers, spaces
    return text.strip()

In [222]:
  # step 11
  not_field = {
    "N/A", "NA", "n/a", "na", "null", "NULL", "", "-", "none", "None",
    "not available", "not applicable", "unspecified", "unknown",
    "not_available", "not_applicable", "not_provided", "not_supplied",
    "not_captured", "not_recorded", "not_collected", "not_entered",
    "not_disclosed", "not_shared", "not_furnished", "not_known",
    "not_identified", "not_determined", "undetermined", "missing",
    "missing_value", "missing_data", "no_data", "no_response",
    "no_answer", "no_result", "empty_value", "blank_value",
    "null_value", "withheld", "redacted", "confidential",
    "pii_removed", "policy_restricted", "compliance_restricted",
    "restricted", "forbidden", "blocked", "safety_blocked",
    "content_blocked", "ambiguous", "unclear", "vague",
    "needs_clarification", "insufficient_context", "insufficient_detail",
    "low_confidence","not provided"
        }

In [223]:
def extract_category(text):
    # Use a regex pattern to capture everything before the first '('
    match = re.match(r"^([^(]+)", text)
    if match:
        return match.group(1).strip()
    else:
        return None 

In [251]:
df_cat.head()

,national_catid,category_name
0,10050521,Body Massage Centres
1,11010608,Massage Centres For Men
2,10272436,Interior Designers
3,10076456,Car Rental
4,10042600,Beauty Spas


In [290]:
import pandas as pd

def search_cat_list(cat_gpt_list):
    df_cat_res = pd.DataFrame()

    for cat in cat_gpt_list:
        # Assuming search_categories returns a DataFrame with columns: ['category', 'cosine_similarity']
        df_cat_temp = search_categories(cat, top_k=5)
        df_cat_res = pd.concat([df_cat_res, df_cat_temp], axis=0)

    # Group by category, count occurrences, and take max cosine similarity
    df_cat_res = (
        df_cat_res.groupby(['national_catid' , 'category_name'], as_index=False)
        .agg(
            category_count=("category_name", "count"),
            cosine_similarity=("cosine_similarity", "max")
        )
    )
    df_cat_res = df_cat_res.sort_values(
    by=['category_count', 'cosine_similarity'], 
    ascending=[False, False]
)
    return df_cat_res


In [319]:
cat_gpt_list = ['Luggage Suppliers', 'Bags Wholesalers', 'Travel Accessories', 'Office Supplies', 'Trolley Bags']

In [291]:
cat_gpt_list =  ["Wedding Caterers", "Catering Services", "Event Caterers", "Party Caterers", "Buffet Caterers"]

In [324]:
cat_gpt_list = ['Edible Oil Suppliers', 'Wholesale Food Suppliers', 'Grocery Suppliers', 'Restaurant Supply Services', 'Cooking Oil Suppliers']

In [325]:
df_cat_res = search_cat_list(cat_gpt_list)

In [326]:
df_cat_res.head()

,national_catid,category_name,category_count,cosine_similarity
11,10237950,Grocery Wholesalers,2,0.890980
12,10285966,Key Manufacturers,2,0.879655
19,10998394,Food Supplement Wholesalers,1,0.902611
6,10179574,Edible Oil Manufacturers,1,0.899967
5,10179559,Edible Oil Distributors,1,0.896461


In [322]:
def main_function(response, area_1="", city_1="", start=False):
    history_conversation = {}
    a = response.lower()
    first_user = input(f"Assistant: {response}\nUser: ")

    if start:
        first_user += f" area is {area_1}, and city is {city_1}. Please use this only for context."

    cnt = 0
    history_conversation[f'{cnt} {a}'] = first_user.lower()
    cnt+=1
    threshold_cat = 0
    threshold_area = 0
    df_cat_res = pd.DataFrame()
    df_area_res = pd.DataFrame()
    cat = area = city = cat_exp = ""

    while cnt < 7:
        # Build prompt and get response
        prompt1 = prompt_func1(history_conversation)
        response = get_response(prompt1)
        parsed = extract_json(response)

        if not parsed:
            user_input = input(f"Assistant: {response}\nUser: ")
            history_conversation[f'{cnt} {response}'] = user_input
            cnt += 1
            continue
        cat_gpt_list = parsed.get("category", "")
        area = preprocess_text(parsed.get("area", ""))
        city = preprocess_text(parsed.get("city", ""))

        # Validate extracted fields
        not_field_clean = {preprocess_text(x) for x in not_field}
        missing_data = []
        # if not cat or cat in not_field_clean:
        #     missing_data.append("Category")
        if not area or area in not_field_clean:
            missing_data.append("Area")
        if not city or city in not_field_clean:
            missing_data.append("City")

        if missing_data:
            response = f"We are unable to get your {', '.join(missing_data)}. Please specify it with city."
            user_input = input(f"Assistant: {response}\nUser: ")
            history_conversation[f'{cnt} {response}'] = user_input
            cnt += 1
            continue

        # Search categories and areas
        df_cat_res = search_cat_list(cat_gpt_list) 
        # df_cat_res = search_categories(cat_exp, top_k=5)
        df_area_res = search_areas(f"{area} {city}", top_k=5)

        if df_cat_res.empty or df_area_res.empty:
            print("DEBUG: One or both DataFrames are empty. Asking user to clarify.")
            response = "We couldn't find relevant matches. Could you clarify?"
            user_input = input(f"Assistant: {response}\nUser: ")
            history_conversation[f'{cnt} {response}'] = user_input
            cnt += 1
            continue

        threshold_cat = df_cat_res['cosine_similarity'].max()
        threshold_area = df_area_res['cosine_similarity'].max()

        print(f"GPT Extracted:\nCategory: {cat_gpt_list}\nArea: {area}\nCity: {city}")

        # Check thresholds
        if threshold_cat < 0.7 or threshold_area < 0.75:
            prompt_question = prompt_func2(
                history_conversation, cat, f"{area} {city}", threshold_cat, threshold_area,
                df_cat_res['category_name'], df_area_res[['main_area', 'city']]
            )
            response = get_response(prompt_question)
            user = input(f"Assistant: {response}\nUser: ")
            history_conversation[response] = user
            cnt += 1
            continue

        # If thresholds satisfied, break loop
        break

    # ✅ Now use gpt_result for final refinement
    if not df_cat_res.empty and not df_area_res.empty:
        # Call GPT to refine category selection
        # cat_exp = extract_category(cat_exp)
        # prompt_cat, category_options = gpt_result(df_cat_res, cat_exp)
        cat_count = df_cat_res['cosine_similarity'].max()
        best_cat_row = df_cat_res[df_cat_res['cosine_similarity']==cat_count]
        # print(f"categpry : {cat_final} , list : {category_options} ")
        # # Fallback if GPT picks something invalid
        # if cat_final not in category_options:
        #     print(f"DEBUG: GPT picked invalid category '{cat_final}'. Falling back to highest similarity.")
        #     if not df_cat_res.empty:
        #         cat_final = df_cat_res.iloc[df_cat_res['cosine_similarity'].idxmax()]['category_name']
        #     else:
        #         print("ERROR: Category DataFrame is empty.")
        #         return None

        # # ✅ Handle category safely
        # best_cat_filtered = df_cat_res[df_cat_res['category_name'] == cat_final]
        # if best_cat_filtered.empty:
        #     print(f"DEBUG: No exact match for '{cat_final}'. Falling back to highest similarity.")
        #     if not df_cat_res.empty:
        #         best_cat_row = df_cat_res.iloc[df_cat_res['cosine_similarity'].idxmax()]
        #     else:
        #         print("ERROR: Category DataFrame is empty.")
        #         return None
        # else:
        #     best_cat_row = best_cat_filtered

        # ✅ Handle area safely
        if not df_area_res.empty:
            best_area_row= df_area_res[df_area_res['cosine_similarity'] == df_area_res['cosine_similarity'].max()]
        else:
            print("ERROR: Area DataFrame is empty.")
            return None

        # Extract final values
        national_catid = best_cat_row['national_catid'].iloc[0]
        category_name = best_cat_row['category_name'].iloc[0]
        main_area = best_area_row['main_area'].iloc[0]
        city_name = best_area_row['city'].iloc[0]
        pincode = best_area_row['pincode'].iloc[0]

        # Output
        output = f"""
The most similar category:
national_catid : {national_catid}
category_name : {category_name}

The most relevant area:
Main_area : {main_area}
Pincode : {pincode}
City : {city_name}
        """
        print(output)
        return national_catid, category_name, city_name, pincode, main_area

    print("Failed to extract proper category and area after multiple attempts.")
    return None


In [17]:
search_categories('Body Massage Centres For Deep Tissue Therapy')

,national_catid,category_name,cosine_similarity
10751,11205802,Body Massage Centres For Deep Tissue Therapy,0.923836
10169,11206702,Deep Tissue Massage Centres,0.915385
0,10050521,Body Massage Centres,0.893054
10420,10050529,Body Massage Centres For Thai Treatment,0.883064
8073,10050569,Body Massage Training Centres,0.882820


In [307]:
  response = 'How can I help you ?'
  area = ''
  city = ''
  start=False

In [323]:
main_function(response, area  , city, start)

Assistant: How can I help you ?
User:  Restaurant in Rajajinagar, Bangalore ke liye edible oil bulk me chahiye. Koi supplier?


GPT Extracted:
Category: ['Edible Oil Suppliers', 'Wholesale Food Suppliers', 'Grocery Suppliers', 'Restaurant Supply Services', 'Cooking Oil Suppliers']
Area: rajajinagar
City: bangalore

The most similar category:
national_catid : 10998394
category_name : Food Supplement Wholesalers

The most relevant area:
Main_area : Rajajinagar
Pincode : 560010
City : Bangalore
        


('10998394', 'Food Supplement Wholesalers', 'Bangalore', 560010, 'Rajajinagar')

In [25]:
search_categories('Brahmin Cook for Marriage' , 25)


,national_catid,category_name,cosine_similarity
9225,10314448,Matrimonial Bureaus For Munnuru Kapu,0.873867
3132,10314411,Matrimonial Bureaus For Brahmin,0.870078
659,10351918,Pandits For Marriage,0.867714
6033,11664879,Brahmin Food Caterers,0.861283
234,10083400,Caterers For Wedding,0.859806
1448,12059295,Stage Decorators For Marriage,0.859163
807,10135275,Cookery Classes,0.857415
8332,10238401,Guest House For Marriage,0.856656
4342,10534743,Wedding Organisers,0.855515
332,10562936,Wedding Planners,0.855270


In [22]:
# step 13
# to continue the conversation with user
def cat_link(category: str, df):
    # Convert both sides to lowercase for comparison
    category_lower = category.lower()
    df['category_name_lower'] = df['category_name'].str.lower()

    # Check if the category exists
    if category_lower in df['category_name_lower'].values:
        # Find the corresponding immidate
        link_cat = df.loc[df['category_name_lower'] == category_lower, 'immidate'].iloc[0]

        # Get all categories with the same immidate
        categories = df.loc[df['immidate'] == link_cat, 'category_name'].tolist()

        return categories, category
    else:
        return [], 'END'  # No match found


In [23]:
import requests

def extract_business_info(api, limit=4):
    try:
        response = requests.get(api, timeout=60)  # disable SSL verify
        response.raise_for_status()
        api_response = response.json()
    except requests.exceptions.Timeout:
        print("⏳ API request timed out.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"⚠️ API request failed: {e}")
        return []

    organic_res = api_response.get("organic_res", {})
    if not isinstance(organic_res, dict):
        return []

    results = []
    for i, (docid, details) in enumerate(organic_res.items()):
        if i >= limit:
            break
        if not isinstance(details, dict):
            continue

        company = details.get("companyname", "")
        city = details.get("city", "")
        area = details.get("area", "")
        rating = details.get("d_rating" , "")
        mobile = details.get("mobile_display", [])
        if isinstance(mobile, (int, float, str)): mobile = [str(mobile)] 
        elif not isinstance(mobile, list): mobile = []
        results.append({
            "docid": docid,
            "companyname": company,
            "mobile_display": mobile,
            "rating" : rating,
            "area": area,
            "city": city
        })

    return results


In [24]:
def clean_text1(x):
    if pd.isna(x):
        return ""
    x = x.lower()
    x = re.sub(r"[^\w]", "", x)  # keep only alphanumeric + underscore
    return x

In [25]:
import pandas as pd

In [26]:
def format_output(data):
    df = pd.DataFrame(data)

    # Ensure required columns exist
    required = ['companyname', 'mobile_display', 'rating', 'area', 'city']
    for c in required:
        if c not in df.columns:
            df[c] = ""

    df = df[required].copy()

    # Normalize mobile_display to a single number (first one if comma-separated)
    def pick_first_mobile(x):
        if isinstance(x, list):
            # take first non-empty element; also split if that element has commas
            for item in x:
                s = str(item).strip()
                if s:
                    return s.split(",")[0].strip()
            return ""
        if pd.isna(x):
            return ""
        return str(x).split(",")[0].strip()

    df['mobile_display'] = df['mobile_display'].apply(pick_first_mobile)
    return df


In [27]:
def search_api(api):
    data = extract_business_info(api, limit=5)
    df = format_output(data)

    # Clean headers & basic formatting
    df = df.rename(columns={
        'companyname': 'Company',
        'mobile_display': 'Mobile',
        'rating': 'Rating',
        'area': 'Area',
        'city': 'City'
    })[['Company', 'Mobile', 'Rating', 'Area', 'City']]

    # Format Rating (keep as string, 1 decimal if numeric)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').map(
        lambda x: f"{x:.1f}" if pd.notnull(x) else ""
    )
    return df

def pretty_print_df(df: pd.DataFrame) -> None:
    # Try nice terminal table
    try:
        from tabulate import tabulate
        print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    except ImportError:
        # Fallback: good-looking pandas print
        with pd.option_context(
            'display.max_colwidth', 60,
            'display.width', 120,
            'display.colheader_justify', 'left'
        ):
            print(df.to_string(index=False))


In [28]:
  response = 'How can I help you ?'
  area = ''
  city = ''
  start=False

In [29]:
def whatshapp_chat():
  end = False
  # Change the path of link categories.
  df = pd.read_excel('link_categories.xlsx')
  response = 'How can I help you ?'
  area = ''
  city = ''
  start=False
  while end!=True:
    try:
      national_catid , cat_name, city_name , pincode , main_area = main_function(response , area , city , start)
      print("ALL OK")  
      start = True
      city = city_name
      area = main_area
      city_name = clean_text1(city_name)
      national_catid = clean_text1(national_catid) 
      api = f'http://192.168.8.12:9001/web_services/CategorySearch_universal.php?catid=&city={city_name}&area={pincode}&start=1&end=10&random1=0.61266&random2=0.34577&random3=0.00138&wap_flag=0&nearme_flag=0&nearme_latitude=&nearme_longitude=&catname=&search_option=1&product_id=&catname_search=&json=1&sort_order=&national_catid={national_catid}&onational_catid=&attribute_values=&state=&darea_flg=0&wap=2&login_mobile=9008008546&pdocid=&trace='  
      data = search_api(api)     # assumes `api` is defined
      pretty_print_df(data)
      temp = input(f"Do you want to end the conversation ? (yes/no): \n").lower()
      if temp == "yes":
        end = True
        break
      categories , cat = cat_link(cat_name , df)
      if cat == 'END':
        print(f'No link categories found with {cat_name}')
        end = True
        break
      prompt2 = prompt_follow1(categories, cat)
      response = get_response(prompt2)
    except Exception as e:
      print(f"An error occurred: {e}")
      end = True

In [124]:
whatshapp_chat()

Assistant: How can I help you ?
User:  Looking for Reiki learning classes around Electronic City, Bangalore for beginners.
Assistant: Could you please confirm the city where you would like to find Reiki learning classes?
User:  Bangalore
Assistant: Could you please specify the area in Bangalore where you are searching for Reiki learning classes?
User:  electronic city
Assistant: Could you please confirm the category of service you are looking for? Are you specifically searching for Reiki learning classes, wellness centers, or something else?
User:  reiki learning service
Assistant: Could you please confirm the category of service you are looking for? Are you specifically searching for Reiki learning classes, wellness centers, or something else?
User:  yes I am looking for Reiki learning classes


GPT Extracted:
Category: Reiki Learning Classes (where individuals learn Reiki healing techniques)
Area: electronic city
City: bangalore
categpry : Reiki Training Services , list : Reiki Training Services, Yoga Classes, Cookery Classes, Reiki Healing Centres, Sketching Classes, Hobby Classes,  

The most similar category:
national_catid : 11568957
category_name : Reiki Training Services

The most relevant area:
Main_area : Electronic City
Pincode : 560100
City : Bangalore
        
ALL OK
+----------------+------------------+----------+--------------------------------+-----------+
| Company        | Mobile           |   Rating | Area                           | City      |
+================+==================+==========+================================+===========+
| Jinal Shah     | +(91)-9538190860 |      5   | Wilson Garden                  | Bangalore |
+----------------+------------------+----------+--------------------------------+-----------+
| Heal Life      | +(91)-9902017039 |

Do you want to end the conversation ? (yes/no): 
 yes


In [121]:
whatshapp_chat()

Assistant: How can I help you ?
User:  Ceiling design and POP false ceiling ka kaam chahiye at my flat in Hebbal, Bangalore.


GPT Extracted:
Category: Ceiling Design and Installation Services (where ceilings are designed and installed)
Area: hebbal
City: bangalore
categpry : Ceiling Interior Designers , list : Construction Services, Ceiling Interior Designers, Building Designers, Building Contractors, Commercial Builders, Interior Designers,  

The most similar category:
national_catid : 11616751
category_name : Ceiling Interior Designers

The most relevant area:
Main_area : Hebbal
Pincode : 560024
City : Bangalore
        
ALL OK
+--------------------------+------------------+----------+------------------------+-----------+
| Company                  | Mobile           |   Rating | Area                   | City      |
+==========================+==================+==========+========================+===========+
| Magswizard Home Solution | +(91)-9538856935 |      5   |                        | Bangalore |
+--------------------------+------------------+----------+------------------------+-----------+
| Build

Do you want to end the conversation ? (yes/no): 
 yes


In [35]:
search_categories('I want catering service for wedding')

,national_catid,categroy_name,cosine_similarity
234,10083400,Caterers For Wedding,0.892742
3349,10083351,Caterers For Events,0.881743
4342,10534743,Wedding Organisers,0.881659
6253,10083352,Caterers For Function,0.881377
7396,10268601,Institutes For Catering,0.878456
